# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
print(song_files)

['/workspace/home/data/song_data/A/B/B/TRABBNP128F932546F.json', '/workspace/home/data/song_data/A/B/B/TRABBXU128F92FEF48.json', '/workspace/home/data/song_data/A/B/B/TRABBZN12903CD9297.json', '/workspace/home/data/song_data/A/B/B/TRABBJE12903CDB442.json', '/workspace/home/data/song_data/A/B/B/TRABBKX128F4285205.json', '/workspace/home/data/song_data/A/B/B/TRABBBV128F42967D7.json', '/workspace/home/data/song_data/A/B/B/TRABBTA128F933D304.json', '/workspace/home/data/song_data/A/B/B/TRABBLU128F93349CF.json', '/workspace/home/data/song_data/A/B/B/TRABBOR128F4286200.json', '/workspace/home/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json', '/workspace/home/data/song_data/A/B/B/TRABBOP128F931B50D.json', '/workspace/home/data/song_data/A/B/B/TRABBAM128F429D223.json', '/workspace/home/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json', '/workspace/home/data/song_data/A/B/C/TRABCEI128F424C983.json', '/workspace/home/data/song_data/A/B/C/TRABCTK128F934B224.json', '/workspace/home/data/song_data/A/B/C/T

In [5]:
filepath = 'data/song_data/A/B/B/TRABBNP128F932546F.json'

In [6]:

df = pd.read_json(filepath,lines=True)
df.head()


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR62SOJ1187FB47BB5,NaN,,NaN,Chase & Status,337.68444,1,SOGVQGJ12AB017F169,Ten Tonne,2005


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id' ,'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005, 337.68444]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['AR62SOJ1187FB47BB5', 'Chase & Status', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')
log_files

['/workspace/home/data/log_data/2018/11/2018-11-04-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-15-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-20-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-12-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-27-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-19-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-25-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-22-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-17-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-28-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-08-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-23-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-16-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-30-events.json',
 '/workspace/home/data/log_data/2018/11/2018-11-13-events.json',
 '/workspace/home/data/lo

In [12]:
filepath = '/workspace/home/data/log_data/2018/11/2018-11-04-events.json'

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,154,None,200,1541290555796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
2,None,Logged In,Jahiem,M,0,Miles,NaN,free,"San Antonio-New Braunfels, TX",GET,Home,1.540817e+12,42,None,200,1541299033796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df.query('page == "NextSong"')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
5,Olivia Ruiz,Logged In,Jahiem,M,3,Miles,254.74567,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,Cabaret Blanco,200,1541300540796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
7,Jordan Rudess,Logged In,Cecilia,F,1,Owens,1367.84934,free,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.541032e+12,225,Tarkus,200,1541306152796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6


In [15]:
t = pd.to_datetime(df['ts'])
t.head()

1   1970-01-01 00:25:41.292603796
3   1970-01-01 00:25:41.300092796
4   1970-01-01 00:25:41.300337796
5   1970-01-01 00:25:41.300540796
7   1970-01-01 00:25:41.306152796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = (t,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday)
column_labels = ('start_time','hour','day','week','month','year','weekday')

In [17]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()


,start_time,hour,day,week,month,year,weekday
1,1970-01-01 00:25:41.292603796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.300092796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.300337796,0,1,1,1,1970,3
5,1970-01-01 00:25:41.300540796,0,1,1,1,1970,3
7,1970-01-01 00:25:41.306152796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
1,99,Ann,Banks,F,free
3,43,Jahiem,Miles,M,free
4,43,Jahiem,Miles,M,free
5,43,Jahiem,Miles,M,free
7,6,Cecilia,Owens,F,free
8,25,Jayden,Graves,M,paid
9,25,Jayden,Graves,M,paid
14,25,Jayden,Graves,M,paid
15,25,Jayden,Graves,M,paid
16,25,Jayden,Graves,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [index+1, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')
('SOGVQGJ1

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.